In [1]:
import SpinDynamics as sd
import numpy as np

import timeit

In [2]:
mT2angfreq = 2.00231930436153*9.27400968e-24/(6.62606957e-34/2/np.pi)/1e12
dr3 = -4*np.pi*1e-7 * (2.0023193043617 * 9.27400968e-24)**2 / (4*np.pi*1e-30)/6.62606957e-34/1e6 * 2e-3 * np.pi #rad/ns A^3

In [13]:
#set the spins, 3e in this case
spins = [0.5, 0.5, 0.5, 0.5]
field = np.array([0, 0, 1])*mT2angfreq
hyp   = np.diag([0.2, 0.2, 0.2])
kS0 = 0.2
beta = 1.4
ksc = 1/100
radius = 4.5 #set the radius of the radicals
coordinates = [[0, 0, 0],
              [9, 0, 0],
              [0, 9, 0],
              None]

#create the particle objects
sp = [sd.Particles.HighMemSpinParticle(spins, i, coordinates[i]) for i in range(len(spins))]

#First 3 particles are e, last 1 is n
Hamiltonian = sum([sd.Hamiltonian.CalculateHamiltonian.zeeman(sp[i], field) for i in range(len(spins)-1)])

#Interaction between the 1st e and 1st n
Hamiltonian += sd.Hamiltonian.CalculateHamiltonian.hyperfine(sp[0], sp[-1], hyp)

#Calculates the distances between all particles, returns a named tuple - much more memory efficient than a dict
dist = sd.CalculateDistances(sp[:-1], range(len(sp)-1))
#Calculating the dipolar contribution between electrons
Hamiltonian += sum([sd.Hamiltonian.CalculateHamiltonian.dipolar(sp[dip.index[0]], sp[dip.index[1]], dip.matrix) 
                    for dip in sd.CalculateDipolarMatrices(dr3, dist)])

#Calculate K1 - Ps (singlet projection) is fed in i.e. CalculateK1(ks, Ps)
K1 = sum([sd.CalculateK1(kS0*np.exp(-beta*(np.linalg.norm(d.distance) - 2*radius)), 
                         sd.SingletProjection(sp[d.index[0]], sp[d.index[1]])) for d in dist])

sd.CalculateYields(Hamiltonian, K1, ksc)

0.8374526872374094

In [1]:
#Benchmark

#Profile code
#Use numba??